### In this notebook we will use Valhalla to snap the latitude/longitude points from VED to roads using OpenStreetMap data.

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import json
import tqdm

Create the headers and the meili request body.

In [2]:
def create_meili_request_body(df_trip):
    """Returns the body for meili for its request.
    """
    coordinates = df_trip.to_json(orient='records')
    head = '{"shape":'
    tail = ""","search_radius": 5, "shape_match":"map_snap", "costing":"auto", "format":"osrm"}"""
    body = head + coordinates + tail
    return body

In [3]:
headers = {'Content-type': 'application/json'}
trace_url = "http://localhost:8002/trace_attributes"

Read all the file names

In [4]:
data_path = "../../VED-master/Data/"
files = [os.path.join(data_path, file) for file in os.listdir(data_path) if file.endswith(".csv")]
files.sort()

The functions that send the requests to the meili
Match types: {'matched': 0, 'interpolated': 1, 'unmatched': 2}

In [5]:
def process_matched_response(matched_points, group_df):
    """ Returns a list of dictionaries for the matched coordinates.
        Dictionary keys: matched_lat, matched_lon, and match_type. 
        If a points was not matched, returns the original coordinates. """
    
    processed_matches = []
    for (i, matched_point) in enumerate(matched_points):
        match_type = {'matched': 0, 'interpolated': 1, 'unmatched': 2}
        
        matched_dict = {'matched_lat': group_df.iloc[i, 1] 
                        if matched_point['type'] == 'unmatched' 
                        else matched_point['lat'], 
                        'matched_lon': group_df.iloc[i, 0] 
                        if matched_point['type'] == 'unmatched' 
                        else matched_point['lon'], 
                        'match_type': match_type[matched_point['type']]}

        processed_matches.append(matched_dict)
    
    return processed_matches
      

In [30]:
def map_match(file):
    """ Takes a csv file and sends requests to Valhalla Meili service to snap the coordinates
    in the file to roads. """
    signal_df = pd.read_csv(file)
    dataframes = []
    
    # Group the dataframe by the columns VehId and Trip
    for _, group_df in signal_df.groupby(['VehId', 'Trip']):
        group = group_df.rename(columns={'Longitude[deg]':'lon','Latitude[deg]':'lat'})[['lon', 'lat']]
        meili_request_body = create_meili_request_body(group)
        r = requests.post(trace_url, data=str(meili_request_body), headers=headers)
        
        if r.status_code == 200:
            res = r.json()
            matched_dicts = process_matched_response(res['matched_points'], group)
            matched_df = pd.DataFrame.from_dict(matched_dicts)
            dataframes.append(matched_df.set_index(group_df.index.copy()))
            
        else:
            print('Error', r.status_code)

    conc_dfs = pd.concat(dataframes)
    signal_df = pd.merge(signal_df, conc_dfs, left_index=True, right_index=True)
    return signal_df

Process all files and save the results to another directory.

In [ ]:
def get_save_path(filename):
    return f"../data/ved-valhalla/{filename[len(data_path)-1:]}"

for file in files:
    matched_df = map_match(file)
    matched_df.to_csv(get_save_path(file), na_rep=np.nan, index=False)